## 프랑스 영화 리뷰 데이터 구축과 감정분석

### By Sungwoo Stanislas Kwon (권성우)

주제 변경 이유:
- 형태소 분석 경험 부족으로 인한 어려움.
- 보여주고자 했던 바를 증명하기에는 부족했던 술어들의 다양성.
- 복잡한 분석 보다는 그런 분석에 사용 될 수 있는 새로운 데이터를 스스로 구축 해보는 것도 의미있는 일 일것 같다고 생각.

새로운 주제:
- 프랑스 텔레비전 산업 관련 사이트 allociné.fr
- 그 중 '영화' 목록에있는 영화들의 제목, 별점, 리뷰 등등 스크랩해와 데이터셋 구축, 감정분석 실시.

주요 태스크:
- BeautifulSoup를 이용한 데이터 스크래핑과 데이터셋 구축, 전처리 & 간단한 수치들 시각화.
- TF-IDF embedding을 이용해 간단한 감정 분류기 구축(로지스틱 회귀).
- BERT?

## 1. 데이터 스크래핑과 전처리
[allociné.fr](https://www.allocine.fr/)에서 십만개가 넘는 영화 url들을 추출하고 그 url에 접근해서 해당 영화 리뷰 (최대) 30개, 별점등을 스크래핑해와 데이터셋 구축
- 데이터셋을 1부터10까지 만들어야 했기에 사실상 가장 복잡했던 부분.
- 또한 처리해야하는 데이터가 상당히 많아서 코드 돌리는데도 시간이 매우 오래걸림.  

In [ ]:
allocine_df.sample(3, random_state=42)

![image.png](project\images\df_example.png)

Polarity coding:
- 1: positive: 4-5점
- 0: neutral: 2점 초과 4점 미만
- -1: negative: 0-2점

Counts of ratings            |  Coded polarity
:-------------------------:|:-------------------------:
![](project\images\rating_counts.png)  |  ![](project\images\coded_polarity.png)

### 년도별 리뷰 개수

![](project\images\review_peryear.png)

### 리뷰 길이 분포

- 2000자 이상으로 구성되어 있는 리뷰(전체 데이터의 약 6%)는 잘라냄.

![](project\images\review_length.png)

In [ ]:
def loss_percentage(df, previous_length):
    new_length = len(df)
    percentage = 100*(1-(new_length/previous_length))    
    return new_length, percentage  

# 1. 필요없는 열 삭제
dataset_df = dataset_df.drop(columns=['rating', 'date', 'helpful', 'unhelpful'])

# 2. 필요없는 행 삭제 (polarity가 neutral한 행들)
dataset_df = dataset_df[dataset_df['polarity'] != 0]
length, percentage = loss_percentage(dataset_df, initial_len)
print("Length: {} (-{:.1f} %)".format(length, percentage))

# 3. 2000자 넘는 리뷰 삭제
LENGTH_THRESH = 2000
dataset_df = dataset_df[dataset_df['review'].str.len() <= LENGTH_THRESH]
length, percentage = loss_percentage(dataset_df, length)
print("Length: {} (-{:.1f} %)".format(length, percentage))

# 4. 각 영화별 리뷰 30개로 제한
MAX_REVIEWS_PER_FILM = 30
grouped = dataset_df.groupby('film-url')
for ids in grouped.groups.values():
    num_reviews = len(ids)    
    if num_reviews > MAX_REVIEWS_PER_FILM:
        sampling_size = num_reviews - MAX_REVIEWS_PER_FILM
        ids_to_drop = random.sample(list(ids), sampling_size)
        dataset_df = dataset_df.drop(ids_to_drop)

length, percentage = loss_percentage(dataset_df, length)
print("Length: {} (-{:.1f} %)".format(length, percentage))

length, percentage = loss_percentage(dataset_df, initial_len)
print("Total loss: (-{:.1f} %)".format(percentage))

그 밖의 전처리

- negative한 리뷰를 -1에서 0으로 코딩
- 텍스트 클리닝 (공백, 줄 바꾸기 제거 등.)
- 비어있는 리뷰 삭제
- 긍정 & 부정 리뷰 개수 맞추기

![](project\images\data_freq.png)

# 2. TF-IDF를 이용한 감정분류 머신러닝 모델 구현

In [ ]:
import pickle
import numpy as np

In [ ]:
PICKLE_PATH = "allocine_dataset.pickle"

with open(PICKLE_PATH, 'rb') as reader:
    data = pickle.load(reader)

X_train, y_train = np.array(data["train_set"]['review']), np.array(data["train_set"]['polarity'])
X_val, y_val = np.array(data["val_set"]['review']), np.array(data["val_set"]['polarity'])
X_test, y_test = np.array(data["test_set"]['review']), np.array(data["test_set"]['polarity'])
class_names = data['class_names']

print("LEN TRAIN: "+ str(len(X_train)))
print("LEN VAL: "+ str(len(X_val)))
print("LEN TEST: "+ str(len(X_test)))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

tfidf_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(n_jobs=-1, verbose=1)),
])

tfidf_clf.fit(X_train, y_train)

In [ ]:
from sklearn import metrics

# Predicting training dataset
y_pred = tfidf_clf.predict(X_train)
print("Training Accuracy:", metrics.accuracy_score(y_train, y_pred))

# Predicting with a test dataset

y_pred = tfidf_clf.predict(X_val)
print("Validation Accuracy:", metrics.accuracy_score(y_val, y_pred))
print()
print(metrics.classification_report(y_val, y_pred, target_names=class_names.values()))

In [ ]:
from sklearn import metrics

# Predicting training dataset
y_pred = tfidf_clf.predict(X_train)
print("Training Accuracy:", metrics.accuracy_score(y_train, y_pred))

# Predicting with a test dataset

y_pred = tfidf_clf.predict(X_val)
print("Validation Accuracy:", metrics.accuracy_score(y_val, y_pred))
print()
print(metrics.classification_report(y_val, y_pred, target_names=class_names.values()))

![](project\images\learning_curve.png) 

In [ ]:
false_pos = X_val[(y_val == 0) & (y_pred == 1)]
false_neg = X_val[(y_val == 1) & (y_pred == 0)]

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

custom_stopwords = set(["de", "se", "à", "en", "par", "le", "la", "les", "aux", "et", "est", "ses", "au", "du", "quelques", "faire", 
                        "ce", "cette", "si", "dans", "un", "des", "film","films","pas","une","que","mai", "son", "qui", "avec","il",
                        "elle","ils","pour","c'est","sont","sur","je","qu'il", "qu'elle", "sa", "tout", "même",'fait','ne','plu','san',
                        'plus','voir','d','mais','très','bien','bon','ou','sans','comme','peu','nous','y','ça','l','scène',"n'est","d'un",
                        "acteur","personnage",'deux','lui','leur','entre','vous','vou',"d'une","aussi","ces","où","scénario","acteurs","dont",
                        "quand",'cela','ceci','personnages','être','peut','moins','rôles','qu','histoire',"j'ai",'car',"qu'on",'reste','toute',
                        'tous','là','vraiment','été','autre'])

# Add default English stopwords from WordCloud
stopwords = set(STOPWORDS)
stopwords.update(custom_stopwords)

# Word cloud for false_pos
false_pos_text = ''.join(false_pos)
wordcloud_false_pos = WordCloud(width=800, height=400, max_words=200, background_color='white', stopwords=stopwords).generate(false_pos_text)

# Plot word cloud for false_pos
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud_false_pos, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for False Positives')
plt.show()

# Word cloud for false_neg
false_neg_text = ''.join(false_neg)
wordcloud_false_neg = WordCloud(width=800, height=400, max_words=200, background_color='white', stopwords=stopwords).generate(false_neg_text)

# Plot word cloud for false_neg
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud_false_neg, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for False Negatives')
plt.show()